# RGTAN-AFF 模型代码详解
-  by 翁振昊

- 基于论文《Enhancing Attribute-driven Fraud Detection with Risk-aware Graph Representation》的RGTAN模型进行的改进。

- 在属性嵌入模块中，引入了注意力特征融合机制，以替代原始模型中对不同类别特征嵌入进行简单相加的方式。

本文档是为了详细解释 `rgtan_aff_model.py` 文件中的主要类和方法,RGTAN相关的实验测试都在文件夹RGTAN_test里

### 导入必要的库

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim # 尽管此文件不直接用，但通常模型文件会包含
from dgl.utils import expand_as_pair
from dgl import function as fn
from dgl.base import DGLError
from dgl.nn.functional import edge_softmax # 用于 TransformerConv
import numpy as np
import pandas as pd # 用于 TransEmbedding 初始化时的df处理 (可选，取决于实际使用)
from math import sqrt

## 1. PosEncoding类：位置编码模块

这个类用于生成位置编码，通常在处理序列数据或时间信息时使用，为模型提供关于位置或顺序的先验知识。在RGTAN的上下文中，它可能用于对交易的时间戳进行编码，但具体应用取决于 `TransEmbedding` 类如何使用它。

**核心**: 利用不同频率的正弦和余弦函数来为不同位置创建独特的编码向量。

In [2]:
class PosEncoding(nn.Module):
    """
    位置编码模块
    """
    def __init__(self, dim, device, base=10000, bias=0):
        """
        初始化位置编码组件
        :param dim: 编码维度
        :param device: 模型训练设备
        :param base: 编码基数，用于计算不同频率
        :param bias: 编码偏差，可以理解为相位的偏移
        """
        super(PosEncoding, self).__init__()
        p = []  # 存储周期的倒数 (1 / base^(2i/dim))
        sft = [] # 存储相移 (0 或 pi/2)
        for i in range(dim):
            # 计算频率相关的指数项
            b = (i - i % 2) / dim 
            p.append(base ** -b)
            # 交替使用 sin 和 cos (通过相移实现)
            if i % 2: # 奇数维度
                sft.append(np.pi / 2.0 + bias) # cos(x) = sin(x + pi/2)
            else: # 偶数维度
                sft.append(bias) # sin(x)
        
        self.device = device
        # 将相移和周期基数转换为tensor并移到指定设备
        self.sft = torch.tensor(sft, dtype=torch.float32).view(1, -1).to(device)
        self.base = torch.tensor(p, dtype=torch.float32).view(1, -1).to(device)

    def forward(self, pos):
        """
        计算给定位置的位置编码
        :param pos: 位置信息，可以是一个标量、列表或tensor
        :return: 位置编码tensor
        """
        with torch.no_grad(): # 位置编码通常是固定的，不参与梯度更新
            if isinstance(pos, list): # 如果输入是列表，转换为tensor
                pos = torch.tensor(pos, dtype=torch.float32).to(self.device)
            pos = pos.view(-1, 1) # 确保pos是列向量 (num_positions, 1)
            
            # 核心公式: sin(pos / base_period + phase_shift)
            # pos / self.base: 对应 w_k * t 中的 w_k * pos 部分
            # self.sft: 对应相位
            x = pos / self.base + self.sft
            return torch.sin(x)

## 2.TransformerConv类：门控时间图注意力层 (GTGA)
这个类是论文中提出的 **门控时间图注意力 (Gated Temporal Graph Attention, GTGA)** 机制的核心实现。它用于在图节点之间传递和聚合信息，是典型的图注意力网络 (GAT) 的变体，并加入了门控机制和残差连接。

**主要功能为：**:
- **多头注意力**: 计算节点间的多头注意力权重，使模型能关注到更重要的邻居。
- **消息聚合**: 根据注意力权重加权聚合邻居节点的特征。
- **门控机制**: 动态地平衡聚合后的信息和节点自身的原始信息。
- **残差连接与层归一化**: 帮助稳定训练过程，缓解梯度消失问题。

*注意：虽然我们的RGTAN-AFF的创新点不在这个类，但为了模型的完整性，它依然是RGTAN架构的一部分。

In [3]:
class TransformerConv(nn.Module):
    """
    图注意力层，改编自DGL的TransformerConv，加入了门控机制等
    对应论文中的 Gated Temporal Graph Attention (GTGA)
    """
    def __init__(self,
                 in_feats,
                 out_feats,
                 num_heads,
                 bias=True,
                 allow_zero_in_degree=False,
                 skip_feat=True, # 是否使用残差连接
                 gated=True,     # 是否使用门控机制
                 layer_norm=True,# 是否使用层归一化
                 activation=nn.PReLU()):
        super(TransformerConv, self).__init__()
        self._in_src_feats, self._in_dst_feats = expand_as_pair(in_feats) # 处理同构图和异构图的输入特征维度
        self._out_feats = out_feats # 每个注意力头的输出特征维度
        self._allow_zero_in_degree = allow_zero_in_degree # 是否允许图中存在入度为0的节点
        self._num_heads = num_heads # 注意力头的数量

        # 线性变换层，用于计算Query, Key, Value
        self.lin_query = nn.Linear(self._in_src_feats, self._out_feats * self._num_heads, bias=bias)
        self.lin_key = nn.Linear(self._in_src_feats, self._out_feats * self._num_heads, bias=bias)
        self.lin_value = nn.Linear(self._in_src_feats, self._out_feats * self._num_heads, bias=bias)

        # 残差连接的线性变换层
        if skip_feat:
            self.skip_feat = nn.Linear(self._in_src_feats, self._out_feats * self._num_heads, bias=bias)
        else:
            self.skip_feat = None
        
        # 门控机制的线性变换层
        if gated:
            # 输入是 skip_feat, rst, skip_feat - rst 拼接而成，所以维度是 3 * out_feats * num_heads
            self.gate = nn.Linear(3 * self._out_feats * self._num_heads, 1, bias=bias)
        else:
            self.gate = None
        
        # 层归一化
        if layer_norm:
            self.layer_norm = nn.LayerNorm(self._out_feats * self._num_heads)
        else:
            self.layer_norm = None
        
        self.activation = activation # 激活函数

    def forward(self, graph, feat, get_attention=False):
        """
        :param graph: DGLGraph对象，当前计算子图
        :param feat: 输入节点特征，形状 (num_nodes, in_feats)
        :param get_attention: 是否返回注意力权重，默认为False
        :return: 输出节点特征，形状 (num_dst_nodes, out_feats * num_heads)
                 如果get_attention为True，则额外返回注意力权重
        """
        with graph.local_scope(): # 使用local_scope确保对图的操作不影响外部
            if not self._allow_zero_in_degree:
                if (graph.in_degrees() == 0).any():
                    raise DGLError('There are 0-in-degree nodes in the graph, '
                                   'output for those nodes will be invalid. ')

            if isinstance(feat, tuple): # 用于异构图，源节点和目标节点特征不同
                h_src = feat[0]
                h_dst = feat[1]
            else: # 用于同构图，源节点和目标节点特征相同
                h_src = feat
                h_dst = h_src[:graph.number_of_dst_nodes()] # 取目标节点的特征

            # 1. 线性变换得到 Q, K, V
            # (num_nodes, in_feats) -> (num_nodes, num_heads * out_feats) -> (num_nodes, num_heads, out_feats)
            q_src = self.lin_query(h_src).view(-1, self._num_heads, self._out_feats)
            k_dst = self.lin_key(h_dst).view(-1, self._num_heads, self._out_feats)
            v_src = self.lin_value(h_src).view(-1, self._num_heads, self._out_feats)
            
            # 将Q和V赋给源节点，K赋给目标节点 (在DGL消息传递中，Q和K来自不同侧)
            graph.srcdata.update({'ft_q': q_src, 'ft_v': v_src})
            graph.dstdata.update({'ft_k': k_dst})
            
            # 2. 计算注意力分数 (Attention Score)
            # 使用DGL的内置函数 apply_edges(fn.u_dot_v(...)) 计算源节点Q和目标节点K的点积
            # e_ij = (q_i * k_j) / sqrt(d_k)
            graph.apply_edges(fn.u_dot_v('ft_q', 'ft_k', 'a')) # 'a' 是边的注意力原始分数
            
            # 3. Softmax归一化得到注意力权重
            # graph.edata['a'] 形状是 (num_edges, num_heads, 1)
            # 除以 sqrt(out_feats) 是Transformer中的缩放因子
            graph.edata['sa'] = edge_softmax(graph, graph.edata['a'] / self._out_feats**0.5)
            
            # 4. 加权聚合邻居信息 (Value)
            # 使用DGL的内置函数 update_all(fn.u_mul_e(...), fn.sum(...))
            # 将源节点的Value (ft_v) 与边上的注意力权重 (sa) 相乘，然后对目标节点的所有入边进行求和
            graph.update_all(fn.u_mul_e('ft_v', 'sa', 'attn_msg'), fn.sum('attn_msg', 'agg_u'))
            
            # 获取目标节点聚合后的特征
            # (num_dst_nodes, num_heads, out_feats) -> (num_dst_nodes, num_heads * out_feats)
            rst = graph.dstdata['agg_u'].reshape(-1, self._out_feats * self._num_heads)

            # 5. 残差连接 (Skip Connection)
            if self.skip_feat is not None:
                skip_feat_transformed = self.skip_feat(h_dst) # 对原始目标节点特征进行线性变换
                
                # 6. 门控机制 (Gated Mechanism)
                if self.gate is not None:
                    # 拼接三种信息用于计算门控值
                    gate_input = torch.cat([skip_feat_transformed, rst, skip_feat_transformed - rst], dim=-1)
                    gate_val = torch.sigmoid(self.gate(gate_input))
                    # 门控加权求和
                    rst = gate_val * skip_feat_transformed + (1 - gate_val) * rst
                else: # 如果没有门控，就是简单的相加
                    rst = skip_feat_transformed + rst
            
            # 7. 层归一化 (Layer Normalization)
            if self.layer_norm is not None:
                rst = self.layer_norm(rst)
            
            # 8. 激活函数
            if self.activation is not None:
                rst = self.activation(rst)

            if get_attention:
                return rst, graph.edata['sa']
            else:
                return rst

## 3.Tabular1DCNN2类：表格数据的一维卷积网络

这个类是论文中用于处理 **邻居风险感知特征** 的一个定制化的一维卷积神经网络。邻居风险感知特征通常是多个数值统计量如不同跳数的邻居度数、风险邻居计数等，可以看作是一个“表格”或一个“多通道的一维信号”。

**主要目的**:
- 从这些结构化的邻居统计特征中提取更深层次的模式。
- 通过卷积操作捕捉特征之间的局部依赖关系。
- 最终输出一个能代表邻居风险结构的嵌入向量。

它包含多个卷积层、批归一化层、激活函数和残差连接，是一个相对复杂的特征提取器。

In [4]:
class Tabular1DCNN2(nn.Module):
    """
    用于处理表格化邻居统计特征的一维卷积网络。
    输入 x 的形状应为 (batch_size, num_neigh_stat_features)
    在内部会 reshape 成 (batch_size, num_neigh_stat_features, 1) 或类似形式进行1D卷积，
    或者直接在 (batch_size, num_neigh_stat_features, embed_dim) 上操作，
    取决于具体的实现方式。这里的实现更像是把 embed_dim 作为“信号长度”。
    """
    def __init__(self,
                 input_dim: int,  # 邻居统计特征的数量 (例如，有多少种不同的统计值)
                 embed_dim: int,  # 每个统计特征希望映射到的嵌入维度 (作为1D卷积的“信号长度”)
                 K: int = 4,      # 卷积通道倍增因子
                 dropout: float = 0.2):
        super().__init__()
        self.input_dim = input_dim # 输入通道数
        self.embed_dim = embed_dim # 可以理解为每个通道的“长度”
        
        # 初始全连接层，用于将输入扩展到一个较大的维度
        self.hid_dim = input_dim * embed_dim * 2 
        self.bn1 = nn.BatchNorm1d(input_dim) # 对原始输入特征进行BN
        self.dropout1 = nn.Dropout(dropout)
        self.dense1 = nn.Linear(input_dim, self.hid_dim) # (B, num_stats) -> (B, hid_dim)

        # 后续卷积操作的参数
        self.cha_input = self.cha_output = input_dim # 保持通道数，但每个通道长度变为 embed_dim * 2
        self.cha_hidden = (input_dim * K) // 2
        self.sign_size1 = 2 * embed_dim # dense1输出reshape后的“信号长度”
        self.sign_size2 = embed_dim     # 池化后的“信号长度”
        self.K = K # 通道倍增因子

        # 第一个卷积块 (分组卷积 + ReLU + 池化)
        self.bn_cv1 = nn.BatchNorm1d(self.cha_input)
        self.conv1 = nn.Conv1d(
            in_channels=self.cha_input,
            out_channels=self.cha_input * self.K, # 通道数增加K倍
            kernel_size=5,
            padding=2,
            groups=self.cha_input, # 分组卷积，每个输入通道独立卷积
            bias=False
        )
        self.ave_pool1 = nn.AdaptiveAvgPool1d(self.sign_size2) # 池化到指定长度

        # 第二个卷积块 (普通卷积 + ReLU + 残差连接)
        self.bn_cv2 = nn.BatchNorm1d(self.cha_input * self.K)
        self.dropout2 = nn.Dropout(dropout)
        self.conv2 = nn.Conv1d(
            in_channels=self.cha_input * self.K,
            out_channels=self.cha_input * self.K, # 通道数不变
            kernel_size=3,
            padding=1,
            bias=True
        )

        # 第三个卷积块
        self.bn_cv3 = nn.BatchNorm1d(self.cha_input * self.K)
        self.conv3 = nn.Conv1d(
            in_channels=self.cha_input * self.K,
            out_channels=self.cha_input * (self.K // 2), # 通道数减半
            kernel_size=3,
            padding=1,
            bias=True
        )

        # 多个残差卷积块
        self.bn_cvs = nn.ModuleList()
        self.convs = nn.ModuleList()
        for i in range(6): # 6个残差块
            self.bn_cvs.append(nn.BatchNorm1d(self.cha_input * (self.K // 2)))
            self.convs.append(nn.Conv1d(
                in_channels=self.cha_input * (self.K // 2),
                out_channels=self.cha_input * (self.K // 2), # 通道数不变
                kernel_size=3,
                padding=1,
                bias=True
            ))

        # 最后的卷积块，将通道数变回初始的 cha_output
        self.bn_cv10 = nn.BatchNorm1d(self.cha_input * (self.K // 2))
        self.conv10 = nn.Conv1d(
            in_channels=self.cha_input * (self.K // 2),
            out_channels=self.cha_output, # 通道数变回 input_dim
            kernel_size=3,
            padding=1,
            bias=True
        )

    def forward(self, x):
        # x 初始形状: (batch_size, input_dim) 代表 num_neigh_stat_features
        
        # 1. 初始全连接和变形
        x = self.dropout1(self.bn1(x))
        x = nn.functional.celu(self.dense1(x)) # (B, hid_dim)
        # 变形以适应1D卷积: (B, cha_input, sign_size1) 
        # cha_input = input_dim, sign_size1 = 2 * embed_dim
        x = x.reshape(x.shape[0], self.cha_input, self.sign_size1) 

        # 2. 第一个卷积块
        x = self.bn_cv1(x)
        x = nn.functional.relu(self.conv1(x)) # (B, cha_input*K, sign_size1)
        x = self.ave_pool1(x) # (B, cha_input*K, sign_size2)

        # 3. 第二个卷积块 (带残差)
        x_input_res = x # 保存残差连接的输入
        x = self.dropout2(self.bn_cv2(x))
        x = nn.functional.relu(self.conv2(x)) # (B, cha_input*K, sign_size2)
        x = x + x_input_res # 残差连接

        # 4. 第三个卷积块
        x = self.bn_cv3(x)
        x = nn.functional.relu(self.conv3(x)) # (B, cha_input*(K//2), sign_size2)

        # 5. 多个残差卷积块
        for i in range(6):
            x_input_res_loop = x
            x = self.bn_cvs[i](x)
            x = nn.functional.relu(self.convs[i](x))
            x = x + x_input_res_loop # 残差连接
        
        # 6. 最后一个卷积层
        x = self.bn_cv10(x)
        x = nn.functional.relu(self.conv10(x)) # (B, cha_output, sign_size2)
                                             # cha_output = input_dim

        # 输出形状: (batch_size, input_dim, embed_dim)
        # 即为每个邻居统计特征都生成了一个 embed_dim 维度的向量
        return x

## 4.TransEmbedding类：交易特征嵌入与融合模块 (核心创新所在)

这个类负责处理一笔交易中的多种原始特征，并将它们转换、融合成统一的向量表示，作为后续图神经网络层的输入。它主要处理三种信息：
1.  **类别属性特征 (Categorical Features)**: 如卡类型、商户ID、交易地点等。
2.  **邻居风险结构特征 (Neighbor Risk Features)**: 由 `Tabular1DCNN2` 处理后的邻居统计特征。
3.  **时间特征 (Time Features)**

**RGTAN-AFF 的核心创新点就在这个类中对“类别属性特征”的处理方式上。**

**原始RGTAN**: 对每个类别特征分别进行嵌入和MLP处理后，直接相加。
**RGTAN-AFF**: 引入了一个**轻量级的注意力网络 (`feature_attention_net`)**。
   - 首先，每个类别特征仍旧分别进行嵌入和独立的MLP处理。
   - 然后，不直接相加，而是将这些处理后的特征嵌入向量输入到 `feature_attention_net` 中，为每个特征计算一个注意力分数。
   - 这些分数经过Softmax归一化后，形成注意力权重。
   - 最后，用这些权重对原始的、经过MLP处理的特征嵌入向量进行加权求和，得到最终融合的类别特征向量 `cat_output`。

**这样做可以：**:
- **动态加权**: 模型可以根据当前交易的上下文，学习到不同类别特征的重要性，并赋予它们不同的权重。
- **更强的表达能力**: 相比简单的无差别相加，注意力机制能更灵活地捕捉特征间的组合关系。
- **潜在的可解释性**: 分析注意力权重可以帮助理解模型在做决策时更依赖哪些原始特征。

In [5]:
class TransEmbedding(nn.Module):
    """
    交易特征嵌入模块，融合了类别特征和邻居风险特征。
    RGTAN-AFF 的创新点在此实现。
    """
    def __init__(self,
                 df=None, # 用于初始化类别特征Embedding时获取唯一值数量
                 device='cpu',
                 dropout=0.2,
                 in_feats_dim=82, # 目标嵌入维度
                 cat_features=None, # 类别特征的列名列表
                 neigh_features: dict = None, # 邻居风险特征的列名或数量 (用于Tabular1DCNN2)
                 att_head_num: int = 4, # 用于邻居风险特征自注意力的头数
                 neighstat_uni_dim=64): # 似乎未使用
        super(TransEmbedding, self).__init__()
        
        # 时间位置编码 (在此类的forward中未直接使用，可能为扩展预留)
        self.time_pe = PosEncoding(dim=in_feats_dim, device=device, base=100)

        # 1. 类别特征嵌入表
        # 为每个类别特征创建一个独立的Embedding层
        # df[col].unique() 用于获取该列唯一值的数量，从而确定Embedding词表大小
        self.cat_table = nn.ModuleDict()
        if df is not None and cat_features is not None:
            self.cat_table = nn.ModuleDict({
                col: nn.Embedding(max(df[col].astype(int).unique()) + 1, in_feats_dim).to(device) 
                for col in cat_features if col not in {"Labels", "Time"}
            })
        
        # 2. 邻居风险特征处理模块
        self.nei_table = None # Tabular1DCNN2
        if isinstance(neigh_features, dict) or isinstance(neigh_features, pd.DataFrame): # 修改判断条件
            num_neigh_stat_features = len(neigh_features.keys()) if isinstance(neigh_features, dict) else len(neigh_features.columns)
            if num_neigh_stat_features > 0:
                 self.nei_table = Tabular1DCNN2(input_dim=num_neigh_stat_features, embed_dim=in_feats_dim)

        # 邻居风险特征的自注意力机制参数 (类似Transformer的自注意力)
        self.att_head_num = att_head_num
        self.att_head_size = int(in_feats_dim / att_head_num) if att_head_num > 0 else 0
        self.total_head_size = in_feats_dim
        self.lin_q = nn.Linear(in_feats_dim * (len(neigh_features.keys()) if neigh_features else 1) , self.total_head_size) # 修改维度以匹配Tabular1DCNN2的输出
        self.lin_k = nn.Linear(in_feats_dim * (len(neigh_features.keys()) if neigh_features else 1), self.total_head_size)
        self.lin_v = nn.Linear(in_feats_dim * (len(neigh_features.keys()) if neigh_features else 1), self.total_head_size)

        self.lin_final = nn.Linear(in_feats_dim, in_feats_dim) # 自注意力输出后的线性层
        self.layer_norm = nn.LayerNorm(in_feats_dim, eps=1e-8) # 层归一化

        # 用于聚合邻居风险特征自注意力输出的MLP
        self.neigh_mlp = nn.Linear(in_feats_dim, 1) # 输出一个标量？论文中是拼接，这里似乎是加权或选择

        # 标签嵌入层 (用于风险传播，在RGTAN主类中使用，这里定义了以备不时之需)
        self.label_table = nn.Embedding(3, in_feats_dim, padding_idx=2).to(device)
        
        self.emb_dict = None # 缓存cat_table
        self.cat_features = cat_features if cat_features else []
        self.neigh_features = neigh_features
        
        # 为每个类别特征创建一个独立的MLP层 (论文图2(b)中的MLP)
        self.forward_mlp = nn.ModuleList(
            [nn.Linear(in_feats_dim, in_feats_dim) for _ in range(len(self.cat_features))]
        )
        self.dropout = nn.Dropout(dropout)
        
        # --- RGTAN-AFF创新点---
        # 定义一个用于计算各类别特征嵌入注意力权重的网络
        # 输入是单个类别特征处理后的嵌入 (维度 in_feats_dim)
        # 输出是一个标量，代表该特征的原始注意力分数
        if self.cat_features: # 仅当有类别特征时才定义
            self.feature_attention_net = nn.Sequential(
                nn.Linear(in_feats_dim, in_feats_dim // 2),
                nn.Tanh(), # 使用Tanh作为激活函数
                nn.Linear(in_feats_dim // 2, 1) # 输出一个标量
            )
        else:
            self.feature_attention_net = None
        # --- RGTAN-AFF创新点---

    def forward_emb(self, cat_feat: dict):
        """辅助函数：获取类别特征的原始嵌入"""
        if self.emb_dict is None: # 缓存，避免重复赋值
            self.emb_dict = self.cat_table
        # cat_feat 是一个字典，key是特征名，value是该特征的ID tensor
        support = {
            col: self.emb_dict[col](cat_feat[col].long()) # 确保输入是LongTensor
            for col in self.cat_features if col in self.emb_dict # 确保列名存在
        }
        return support

    def transpose_for_scores(self, input_tensor):
        """辅助函数：变形张量以适应多头注意力计算 (B, L, H*D) -> (B, H, L, D)"""
        new_x_shape = input_tensor.size()[:-1] + (self.att_head_num, self.att_head_size)
        input_tensor = input_tensor.view(*new_x_shape)
        return input_tensor.permute(0, 2, 1, 3)

    def forward_neigh_emb(self, neighstat_feat: dict):
        """处理邻居风险统计特征"""
        if not self.nei_table or not neighstat_feat: # 检查 nei_table 是否初始化
            return None, None

        cols = list(neighstat_feat.keys())
        # 将字典中的tensor堆叠起来，形状 (batch_size, num_neigh_stats)
        tensor_list = [neighstat_feat[col] for col in cols]
        neis_input_to_cnn = torch.stack(tensor_list, dim=1) 
        
        # 1. 通过 Tabular1DCNN2 提取深度特征
        # 输入 (B, num_stats), 输出 (B, num_stats, embed_dim)
        input_tensor_for_attention = self.nei_table(neis_input_to_cnn) 
        
        reshaped_input_tensor = input_tensor_for_attention.view(input_tensor_for_attention.size(0), -1)

        mixed_q_layer = self.lin_q(reshaped_input_tensor) # (B, total_head_size)
        mixed_k_layer = self.lin_k(reshaped_input_tensor) 
        mixed_v_layer = self.lin_v(reshaped_input_tensor) 

        q_layer = self.transpose_for_scores(mixed_q_layer.unsqueeze(1)) # (B, num_heads, 1, att_head_size)
        k_layer = self.transpose_for_scores(mixed_k_layer.unsqueeze(1))
        v_layer = self.transpose_for_scores(mixed_v_layer.unsqueeze(1)) 

        att_scores = torch.matmul(q_layer, k_layer.transpose(-1, -2)) # (B, num_heads, 1, 1)
        att_scores = att_scores / sqrt(self.att_head_size if self.att_head_size > 0 else 1.0) # 缩放

        att_probs = nn.Softmax(dim=-1)(att_scores) # (B, num_heads, 1, 1)
        context_layer = torch.matmul(att_probs, v_layer) # (B, num_heads, 1, att_head_size)
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous() # (B, 1, num_heads, att_head_size)
        new_context_shape = context_layer.size()[:-2] + (self.total_head_size,) # (B, 1, total_head_size)
        context_layer = context_layer.view(*new_context_shape).squeeze(1) # (B, total_head_size)
        
        hidden_states = self.lin_final(context_layer) 
        hidden_states = self.layer_norm(hidden_states)

        return hidden_states, cols


    def forward(self, cat_feat: dict, neighstat_feat: dict):
        """
        前向传播函数
        :param cat_feat: 字典，包含各类别特征的ID, e.g., {'feat1': tensor([...]), 'feat2': tensor([...])}
        :param neighstat_feat: 字典，包含各邻居统计特征的值, e.g., {'deg1': tensor([...]), 'risk1': tensor([...])}
        :return: cat_output (融合后的类别特征嵌入), nei_output (处理后的邻居风险特征嵌入)
        """
        # 1. 处理类别特征
        cat_output = None
        if self.cat_features and self.feature_attention_net and cat_feat:
            support = self.forward_emb(cat_feat) # 获取原始嵌入
            
            processed_embeddings = []
            valid_feature_keys = [] # 存储实际处理的特征键，以防cat_feat不完整
            for i, k_col_name in enumerate(self.cat_features): # 遍历预定义的类别特征顺序
                if k_col_name in support:
                    emb = support[k_col_name]
                    # 每个类别特征先经过各自的MLP层
                    # forward_mlp的索引应与cat_features的索引对应
                    processed_emb = self.dropout(emb)
                    processed_emb = self.forward_mlp[i](processed_emb)
                    processed_embeddings.append(processed_emb)
                    valid_feature_keys.append(k_col_name)

            if processed_embeddings:
                # (batch_size, num_valid_cat_features, embedding_dim)
                stacked_embeddings = torch.stack(processed_embeddings, dim=1)
                
                # 计算每个特征的注意力分数
                # (B, N_valid, D) -> (B, N_valid, 1)
                attention_scores = self.feature_attention_net(stacked_embeddings)
                
                # Softmax归一化得到权重 (在特征维度 N_valid 上)
                attention_weights = torch.softmax(attention_scores, dim=1)
                
                # 加权求和
                # (B, N_valid, D) * (B, N_valid, 1) -> 广播，逐元素相乘
                # 然后在特征维度(dim=1)上求和，得到 (B, D)
                cat_output = torch.sum(stacked_embeddings * attention_weights, dim=1)
            else: # 如果没有有效的类别特征被处理
                bs = 0
                if cat_feat:
                    first_key = next(iter(cat_feat))
                    bs = cat_feat[first_key].size(0)
                elif neighstat_feat:
                    first_key = next(iter(neighstat_feat))
                    bs = neighstat_feat[first_key].size(0)
                
                if bs > 0 :
                    cat_output = torch.zeros(bs, self.in_feats_dim).to(self.device if hasattr(self, 'device') else 'cpu')
                # else: cat_output 保持 None，或者根据后续逻辑处理

        # 2. 处理邻居风险特征
        nei_output_final = None # 初始化
        if neighstat_feat and self.neigh_features : # 确保 neighstat_feat 和 self.neigh_features 都有效
            nei_embs, _ = self.forward_neigh_emb(neighstat_feat) 
            if nei_embs is not None:
                nei_output_final = nei_embs 
        
        # 如果其中一个为空，需要提供一个零向量作为占位符，以确保后续操作（如拼接或相加）的维度正确
        # 获取批大小 (batch_size)
        current_batch_size = 0
        if cat_output is not None:
            current_batch_size = cat_output.size(0)
        elif nei_output_final is not None:
            current_batch_size = nei_output_final.size(0)
        else: # 如果两者都为None，尝试从输入特征获取bs
            if cat_feat:
                first_key = next(iter(cat_feat))
                current_batch_size = cat_feat[first_key].size(0)
            elif neighstat_feat: # 确保neighstat_feat不为空字典
                first_key = next(iter(neighstat_feat))
                current_batch_size = neighstat_feat[first_key].size(0)
        
        # 如果 cat_output 是 None (例如没有类别特征或处理失败)，则创建零向量
        if cat_output is None and current_batch_size > 0:
            cat_output = torch.zeros(current_batch_size, self.in_feats_dim).to(self.device if hasattr(self, 'device') else 'cpu')

        # 如果 nei_output_final 是 None (例如没有邻居特征或处理失败)，则创建零向量
        if nei_output_final is None and current_batch_size > 0:
            # 邻居特征的输出维度应与 cat_output 一致，即 in_feats_dim
            nei_output_final = torch.zeros(current_batch_size, self.in_feats_dim).to(self.device if hasattr(self, 'device') else 'cpu')
            pass 

        return cat_output, nei_output_final

## 5. RGTAN类：模型主体架构

这是实现 RGTAN (或 RGTAN-AFF) 完整功能的模型主类。它整合了前面定义的各个模块：
- `TransEmbedding`: 用于获取初始的节点特征嵌入（包含类别特征融合和邻居风险特征）。
- 标签嵌入 (Risk Propagation): 将交易标签也作为一种特征嵌入，并与节点属性融合。
- 多层 `TransformerConv` (GTGA): 进行图上的消息传递和节点表示学习。
- 最终的输出层: 一个MLP，用于基于学习到的节点表示进行欺诈预测。

**RGTAN-AFF 与 RGTAN 的主要区别在于 `TransEmbedding` 模块的内部实现，而 `RGTAN` 这个主类的整体架构和流程保持不变。**

In [6]:
class RGTAN(nn.Module):
    """
    RGTAN 模型主体。
    如果 TransEmbedding 是 RGTAN-AFF 版本，则此模型即为 RGTAN-AFF。
    """
    def __init__(self,
                 in_feats, # 原始数值特征的维度
                 hidden_dim, # GNN隐层中每个头的维度
                 n_layers, # GNN层数
                 n_classes, # 输出类别数 (欺诈检测通常是2)
                 heads, # GNN每层的注意力头数，列表形式，e.g., [4, 4] for 2 layers
                 activation,
                 skip_feat=True, # TransformerConv参数：是否用残差
                 gated=True,     # TransformerConv参数：是否用门控
                 layer_norm=True,# TransformerConv参数：是否用层归一化
                 post_proc=True, # 是否在GNN输出后接一个后处理MLP
                 n2v_feat=True,  # 是否使用 TransEmbedding 处理类别和邻居特征
                 drop=None,      # Dropout率，列表形式 [input_dropout, hidden_dropout]
                 ref_df=None,    # 用于 TransEmbedding 初始化 cat_table
                 cat_features=None, # 类别特征列名列表
                 neigh_features=None, # 邻居风险特征信息
                 nei_att_head=4,    # TransEmbedding中邻居风险自注意力的头数
                 device='cpu'):
        super(RGTAN, self).__init__()
        self.in_feats = in_feats # 原始数值特征维度
        self.hidden_dim = hidden_dim # 注意力头的基础维度
        self.n_layers = n_layers
        self.n_classes = n_classes
        self.heads = heads # GNN每层的头数列表
        self.activation = activation
        self.input_drop = nn.Dropout(drop[0]) if drop else nn.Identity()
        self.drop = drop[1] if drop and len(drop) > 1 else 0.0
        self.output_drop = nn.Dropout(self.drop)
        self.device = device # 将device保存为成员变量

        # 初始化 TransEmbedding 模块
        self.n2v_mlp = None
        self.actual_in_feats_for_gnn = in_feats # GNN的实际输入维度，会根据是否有TransEmbedding的输出调整
        self.nei_feat_dim_from_trans_embedding = 0 # 从TransEmbedding来的邻居特征维度

        if n2v_feat:
            self.n2v_mlp = TransEmbedding(
                df=ref_df, 
                device=device, 
                in_feats_dim=in_feats, # TransEmbedding内部输出的cat_embed维度与原始数值特征维度一致
                cat_features=cat_features, 
                neigh_features=neigh_features, 
                att_head_num=nei_att_head
            )

            if neigh_features: # 假设如果用邻居特征，则会拼接
                 self.actual_in_feats_for_gnn = in_feats + in_feats # X+cat_embed 和 neigh_embed 拼接
                 self.nei_feat_dim_from_trans_embedding = in_feats 
            else:
                 self.actual_in_feats_for_gnn = in_feats # 只有 X+cat_embed
        else: # 如果不使用TransEmbedding，GNN输入就是原始数值特征
            self.actual_in_feats_for_gnn = in_feats


        self.layers = nn.ModuleList()
        
        # 1. 标签嵌入层 (Risk Propagation)

        self.layers.append(nn.Embedding(
            n_classes + 1, self.actual_in_feats_for_gnn, padding_idx=n_classes))
        
        # 2. 融合标签嵌入与节点特征的MLP (论文图2(a)中的融合部分)
        # 论文公式: h_tilde = MLP_node(h) + MLP_label(h_L_bar)

        output_dim_for_label_fusion_mlp = self.hidden_dim * self.heads[0]

        self.layers.append(nn.Linear(self.actual_in_feats_for_gnn, output_dim_for_label_fusion_mlp))
        self.layers.append(nn.Linear(self.actual_in_feats_for_gnn, output_dim_for_label_fusion_mlp)) # 标签嵌入的输出维度也是actual_in_feats_for_gnn
        self.layers.append(nn.Sequential(
            nn.BatchNorm1d(output_dim_for_label_fusion_mlp),
            nn.PReLU(),
            nn.Dropout(self.drop),
            nn.Linear(output_dim_for_label_fusion_mlp, self.actual_in_feats_for_gnn) # 映射回h的维度
        ))

        # 3. GNN层 (多层TransformerConv)
        # 第一层GNN
        self.layers.append(TransformerConv(
            in_feats=self.actual_in_feats_for_gnn, # 经过特征融合和标签嵌入后的维度
            out_feats=self.hidden_dim, # 每个头的输出维度
            num_heads=self.heads[0],
            skip_feat=skip_feat,
            gated=gated,
            layer_norm=layer_norm,
            activation=self.activation
        ))
        
        # 后续GNN层
        # 输入维度是上一层GNN的输出维度 (hidden_dim * num_heads)
        for l in range(1, self.n_layers): # 注意循环从1开始
            self.layers.append(TransformerConv(
                in_feats=self.hidden_dim * self.heads[l-1], 
                out_feats=self.hidden_dim,
                num_heads=self.heads[l],
                skip_feat=skip_feat,
                gated=gated,
                layer_norm=layer_norm,
                activation=self.activation
            ))
        
        # 4. 输出层 (Post-processing MLP)
        if post_proc:
            self.layers.append(nn.Sequential(
                nn.Linear(self.hidden_dim * self.heads[-1], self.hidden_dim * self.heads[-1]),
                nn.BatchNorm1d(self.hidden_dim * self.heads[-1]),
                nn.PReLU(),
                nn.Dropout(self.drop),
                nn.Linear(self.hidden_dim * self.heads[-1], self.n_classes)
            ))
        else:
            self.layers.append(nn.Linear(self.hidden_dim * self.heads[-1], self.n_classes))

    def forward(self, blocks, features, labels, n2v_feat=None, neighstat_feat=None):
        """
        :param blocks: DGLGraph blocks for message passing in each GNN layer
        :param features: 原始数值特征 (batch_nodes, in_feats)
        :param labels: 节点标签 (用于风险传播) (batch_nodes,)
        :param n2v_feat: 类别特征字典 (给TransEmbedding)
        :param neighstat_feat: 邻居风险特征字典 (给TransEmbedding)
        :return: logits (batch_target_nodes, n_classes)
        """
        h = features # 初始 h 为原始数值特征 (X)

        # 1. 通过TransEmbedding获取类别嵌入(cat_h)和邻居风险嵌入(nei_h)
        if self.n2v_mlp is not None:
            cat_h_from_trans, nei_h_from_trans = self.n2v_mlp(n2v_feat, neighstat_feat)
            
            if cat_h_from_trans is not None:
                h = h + cat_h_from_trans # h = X + hA (论文中的 h_A)
            
            if nei_h_from_trans is not None: # 论文中是拼接 h_N
                if h.size(-1) + nei_h_from_trans.size(-1) == self.actual_in_feats_for_gnn : # 假设拼接后的维度等于GNN输入维度
                     h = torch.cat([h, nei_h_from_trans], dim=-1)
                elif h.size(-1) == nei_h_from_trans.size(-1) : #如果维度相同，相加也是一种选择
                     h = h + nei_h_from_trans # 这种方式意味着actual_in_feats_for_gnn计算时没有增加维度
                else: # 维度不匹配，且不符合预期的拼接后维度，可能需要警告或错误
                    if nei_h_from_trans.size(-1) == self.nei_feat_dim_from_trans_embedding and self.nei_feat_dim_from_trans_embedding > 0 :
                         h = torch.cat([h, nei_h_from_trans], dim=-1)


        # 2. 风险传播 (标签嵌入)
        # self.layers[0] 是 nn.Embedding for labels
        # self.layers[1,2,3] 是用于融合的MLP
        label_embed_raw = self.input_drop(self.layers[0](labels)) # (B, actual_in_feats_for_gnn)
        
        fused_label_info = self.layers[1](h) + self.layers[2](label_embed_raw)
        # 再过一层MLP
        label_embed_processed = self.layers[3](fused_label_info) # (B, actual_in_feats_for_gnn)
        h = h + label_embed_processed

        # 3. 通过多层GNN (TransformerConv)
        # GNN层从 self.layers[4] 开始
        for l in range(self.n_layers):
            # blocks[l] 是第l层GNN对应的计算子图
            # self.layers[l+4] 是第l个TransformerConv层
            h = self.output_drop(self.layers[l+4](blocks[l], h))
            # h 的维度在第一层GNN后变为 (B, hidden_dim * heads[0])
            # 后续层输入输出维度相应变化

        # 4. 最终输出层
        # self.layers[-1] 是最后的MLP分类器
        logits = self.layers[-1](h) # (B, n_classes)

        return logits

In [1]:
from nbconvert import HTMLExporter
import nbformat

# 加载notebook文件
with open('RGTAN-AFF_model.ipynb') as f:
    nb = nbformat.read(f, as_version=4)

html_exporter = HTMLExporter()
html, resources = html_exporter.from_notebook_node(nb)

# 写入HTML文件
with open('RGTAN-AFF_model.html', 'w') as f:
    f.write(html)